<a href="https://colab.research.google.com/github/bm23abb/7PAM2002-0509-2023---Data-Science-Project/blob/main/7PAM2002_0509_2023_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import BayesianRidge

In [19]:
file_path = '/content/drive/MyDrive/7PAM2002-0509-2023-Data-Science-Project/Data-Science-Project/Bitcoin.csv'
data_frame = pd.read_csv(file_path)

In [21]:
print("First few rows of the dataset:")
print(data_frame.head())

First few rows of the dataset:
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  Year  Month  Day  DayOfWeek  
Date                                               
2014-09-17  21056800  2014      9   17          2  
2014-09-18  34483200  2014      9   18          3  
2014-09-19  37919700  2014      9   19          4  
2014-09-20  36863600  2014      9   20          5  
2014-09-21  26580100  2014      9   21          6  


In [20]:
data_frame['Date'] = pd.to_datetime(data_frame['Date'])
data_frame.set_index('Date', inplace = True)
data_frame = data_frame.sort_values('Date')

data_frame = data_frame.fillna(method = 'ffill')

data_frame['Year'] = data_frame.index.year
data_frame['Month'] = data_frame.index.month
data_frame['Day'] = data_frame.index.day
data_frame['DayOfWeek'] = data_frame.index.dayofweek

pforp = data_frame['Close'].values.reshape(-1,1)

In [22]:
normalize = MinMaxScaler(feature_range=(0 ,1))
normalize_price = normalize.fit_transform(pforp)